In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from gensim.models import Word2Vec
from collections import defaultdict
import sys
sys.path.append('../src')
from models import filter_terms_not_in_wemodel, \
    get_2ndorder_association_metric_list_for_target_list, \
    get_1storder_association_metric_list_for_target_list, \
    get_expSG_1storder_relation_no_cache_NEW, \
    get_expSG_1storder_relation_no_cache_NEW_ALLWORDS, \
    get_matrices_from_term_lists, \
    save_array, open_pickle, save_pickle

In [2]:
we_model_name = "sg_dim300_min100_win5"
we_vector_size = 300
we_model_dir = '../data/external/wiki-english/wiki-english-20171001/%s' % we_model_name
we_model = Word2Vec.load(we_model_dir+'/model.gensim')
print ('loading done!')

loading done!


In [3]:
FILEPATH = '../data/interim/association_metric_exps.pickle'

# Define the Experiments

In [4]:
# WEAT 3
EXP_NUM = 3
X_terms = ['roger','alan','frank','ian','justin',
          'ryan','andrew','fred','jack','matthew','stephen','brad','greg','jed',
          'paul','todd','brandon','hank','jonathan','peter','wilbur','amanda',
          'courtney','heather','melanie','sara','amber','crystal','katie',
          'meredith','shannon','betsy','donna','kristin','nancy','stephanie',
          'bobbie-sue','ellen','lauren','peggy','sue-ellen','colleen','emily',
          'megan','rachel','wendy']
Y_terms = ['alonzo','jamel','lerone','theo','alphonse','jerome','leroy',
           'torrance','darnell','lamar','lionel','rashaun','tyree','deion',
          'lamont','malik','terrence','tyrone','lavon','terryl',
          'wardell','aiesha','lashelle','nichelle','shereen','temeka','ebony',
          'latisha','shaniqua','tameisha','teretha','jasmine','laronya','shanise',
          'tanisha','tia','lakisha','latoya','sharise','tashika','yolanda',
          'lashandra','malika','shavonn','tawanda','yvette']
A_terms = ['caress','freedom','health','love','peace','cheer','friend','heaven',
          'loyal','pleasure','diamond','gentle','honest','lucky','rainbow',
          'diploma','gift','honor','miracle','sunrise','family','happy','laughter',
          'paradise','vacation']
B_terms = ['abuse','crash','filth','murder','sickness','accident','death','grief',
          'poison','stink','assault','disaster','hatred','pollute','tragedy','bomb',
          'divorce','jail','poverty','ugly','cancer','evil','kill','rotten','vomit']
X_terms, Y_terms = filter_terms_not_in_wemodel(we_model, X_terms, Y_terms)
A_terms, B_terms = filter_terms_not_in_wemodel(we_model, A_terms, B_terms)

The following terms were removed from the list first_list because they were not found in the we_model: ['bobbie-sue', 'sue-ellen']
The following terms were removed from the list second_list because they were not found in the we_model: ['terryl', 'aiesha', 'lashelle', 'temeka', 'tameisha', 'teretha', 'laronya', 'shanise', 'lakisha', 'sharise', 'tashika', 'lashandra', 'shavonn']
The following terms were removed from the first list to balance the length of the lists: ['roger', 'alan', 'frank', 'ian', 'justin', 'ryan', 'andrew', 'fred', 'jack', 'matthew', 'stephen']
The following terms were removed from the list first_list because they were not found in the we_model: []
The following terms were removed from the list second_list because they were not found in the we_model: []


# First-Order

In [5]:
def get_1storder_association_metric_fast(word, A_terms, B_terms, we_model, scaler):
    A_relations = get_expSG_1storder_relation_no_cache_NEW(word, A_terms, we_model)
    B_relations = get_expSG_1storder_relation_no_cache_NEW(word, B_terms, we_model)
    A_transformed = scaler.transform(A_relations.reshape(-1,1))
    B_transformed = scaler.transform(B_relations.reshape(-1,1))
    return np.mean(A_transformed) - np.mean(B_transformed)

def get_all_relations_1storder(A_terms, B_terms, we_model):
    A_relations=get_expSG_1storder_relation_no_cache_NEW_ALLWORDS(A_terms, we_model)
    B_relations=get_expSG_1storder_relation_no_cache_NEW_ALLWORDS(B_terms, we_model)
    A_associations = np.mean(A_relations, axis=1)
    B_associations = np.mean(B_relations, axis=1)
    
    scaler = MinMaxScaler(feature_range=(-1,1))
    scaler.fit(np.concatenate((A_associations, B_associations))) #concatenation
    return all_associations, scaler

def get_1storder_association_metric_list_for_target_list(target_list, A_terms, B_terms, we_model):
    global ORDER
    ORDER = 'first'
    all_associations, scaler = get_all_relations_1storder(A_terms, B_terms, we_model)
    associations = np.array([])
    for word in tqdm(target_list):
        association = get_1storder_association_metric_fast(word, A_terms, B_terms, we_model, scaler)
        associations = np.append(associations, association)
    
    return associations
    #scaler.fit(all_associations.reshape(-1,1)) # Reshape is for a single feature, NOT for a single sample
    #transformed = scaler.transform(associations.reshape(-1,1))
    #return transformed.reshape(len(transformed))

In [6]:
metrics = get_1storder_association_metric_list_for_target_list(X_terms, A_terms, B_terms, we_model)
# Make sure the last argument passed to save_array matches the first argument passed to
# get_1storder_association_metric_list_for_target_list
save_array(FILEPATH, metrics, EXP_NUM, 'first', 'X')

ValueError: Expected 2D array, got 1D array instead:
array=[0.10841331 0.11889662 0.12435763 ... 0.00466087 0.01104624 0.00433315].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

# Second-Order

In [7]:
def calculate_cosines_for_target_word_unscaled(word_vec, A_mtx, B_mtx):
    A_dot_v = np.dot(A_mtx, word_vec)
    B_dot_v = np.dot(B_mtx, word_vec)
    A_norms = np.multiply(np.linalg.norm(A_mtx, axis=1), np.linalg.norm(word_vec))
    B_norms = np.multiply(np.linalg.norm(B_mtx, axis=1), np.linalg.norm(word_vec))
    A_cosines = np.divide(A_dot_v, A_norms)
    B_cosines = np.divide(B_dot_v, B_norms)
    return np.mean(A_cosines), np.mean(B_cosines)

def get_scaler(we_model, A_mtx, B_mtx):
    '''Computes the association metric, s(w,A,B).
    word_vec: 1-D word vector
    A_mtx, B_mtx: 2-D word vector arrays'''
    #A_cosines_apply = np.apply_along_axis(lambda row: 1-cosine_distance(row, word_vec), 1, A_mtx)
    #B_cosines_apply = np.apply_along_axis(lambda row: 1-cosine_distance(row, word_vec), 1, B_mtx)
    all_words_mtx = we_model.wv.vectors
    all_associations_to_A = np.array([])
    all_associations_to_B = np.array([])
    for x_vec in tqdm(all_words_mtx, desc='Getting association metric for all words'):
        A_cosine, B_cosine = calculate_cosines_for_target_word_unscaled(x_vec, A_mtx, B_mtx)
        all_associations_to_A = np.append(all_associations_to_A, A_cosine)
        all_associations_to_B = np.append(all_associations_to_B, B_cosine)
    scaler = MinMaxScaler()
    scaler = MinMaxScaler(feature_range=(-1,1))
    scaler.fit(np.concatenate((all_associations_to_A, all_associations_to_B)))
    return scaler

def calculate_association_metric_for_target_word_scaled(word_vec, A_mtx, B_mtx, scaler):
    '''Computes the association metric, s(w,A,B).
    word_vec: 1-D word vector
    A_mtx, B_mtx: 2-D word vector arrays'''
    #A_cosines_apply = np.apply_along_axis(lambda row: 1-cosine_distance(row, word_vec), 1, A_mtx)
    #B_cosines_apply = np.apply_along_axis(lambda row: 1-cosine_distance(row, word_vec), 1, B_mtx)
    A_dot_v = np.dot(A_mtx, word_vec)
    B_dot_v = np.dot(B_mtx, word_vec)
    A_norms = np.multiply(np.linalg.norm(A_mtx, axis=1), np.linalg.norm(word_vec))
    B_norms = np.multiply(np.linalg.norm(B_mtx, axis=1), np.linalg.norm(word_vec))
    A_cosines = np.divide(A_dot_v, A_norms)
    B_cosines = np.divide(B_dot_v, B_norms)
    A_transformed = scaler.transform(A_cosines.reshape(-1,1))
    B_transformed = scaler.transform(B_cosines.reshape(-1,1))
    return np.mean(A_transformed) - np.mean(B_transformed)

def get_2ndorder_association_metric_list_for_target_list(target_list, A_terms, B_terms, we_model):
    global ORDER
    ORDER = 'second'
    
    [X_mtx, _, A_mtx, B_mtx] = get_matrices_from_term_lists(we_model, target_list, target_list, A_terms, B_terms)
    associations = np.apply_along_axis(lambda x_vec: calculate_association_metric_for_target_word(x_vec, A_mtx, B_mtx), 1, X_mtx)
    
    scaler = get_scaler(we_model, A_mtx, B_mtx)
    
    associations = np.array([])
    for word in tqdm(target_list):
        association = get_1storder_association_metric_fast(word, A_terms, B_terms, we_model, scaler)
        associations = np.append(associations, association)
    return associations


In [11]:
metrics = get_2ndorder_association_metric_list_for_target_list(X_terms, A_terms, B_terms, we_model)
# Make sure the last argument passed to save_array matches the first argument passed to
# get_1storder_association_metric_list_for_target_list
save_array(FILEPATH, metrics, EXP_NUM, 'second', 'X')

Getting association metric for all words: 100%|██████████████████████████████| 312425/312425 [02:50<00:00, 1830.72it/s]


Results array successfully saved to file ../data/interim/association_metric_exps.pickle under keys [3]['second_order_X']


# Retrieving Results for Plotting

In [12]:
in_dict = open_pickle(FILEPATH)
in_dict

defaultdict(dict,
            {3: {'first_order_X': array([0.10917358, 0.11447405, 0.03977664, 0.21711674, 0.02069784,
                     0.1428209 , 0.23171016, 0.10893403, 0.22164942, 0.1731362 ,
                     0.03183159, 0.08439055, 0.13934323, 0.11403528, 0.10248176,
                     0.15985223, 0.36892206, 0.13024949, 0.11066183, 0.19458479,
                     0.17395213, 0.18826508, 0.09246605, 0.14566013, 0.14104251,
                     0.12317574, 0.17753855, 0.17748386, 0.11440848, 0.06771086,
                     0.07495613, 0.06568201, 0.12313557]),
              'second_order_X': array([0.23449796, 0.2345688 , 0.0989702 , 0.37882134, 0.02910594,
                     0.27740997, 0.16379115, 0.19063637, 0.41707927, 0.29391664,
                     0.1971741 , 0.21215774, 0.3162869 , 0.2859519 , 0.3024328 ,
                     0.18131642, 0.4094162 , 0.2760841 , 0.23804706, 0.30866247,
                     0.30504382, 0.30536607, 0.20394868, 0.3266883 , 0.2867

In [9]:
# OLD RESULTS Before algorithm modification - don't delete
in_dict = open_pickle(FILEPATH)
in_dict

defaultdict(dict,
            {3: {'first_order_X': array([0.10917358, 0.11447405, 0.03977664, 0.21711674, 0.02069784,
                     0.1428209 , 0.23171016, 0.10893403, 0.22164942, 0.1731362 ,
                     0.03183159, 0.08439055, 0.13934323, 0.11403528, 0.10248176,
                     0.15985223, 0.36892206, 0.13024949, 0.11066183, 0.19458479,
                     0.17395213, 0.18826508, 0.09246605, 0.14566013, 0.14104251,
                     0.12317574, 0.17753855, 0.17748386, 0.11440848, 0.06771086,
                     0.07495613, 0.06568201, 0.12313557]),
              'second_order_X': array([0.23449796, 0.2345688 , 0.0989702 , 0.37882134, 0.02910594,
                     0.27740997, 0.16379115, 0.19063637, 0.41707927, 0.29391664,
                     0.1971741 , 0.21215774, 0.3162869 , 0.2859519 , 0.3024328 ,
                     0.18131642, 0.4094162 , 0.2760841 , 0.23804706, 0.30866247,
                     0.30504382, 0.30536607, 0.20394868, 0.3266883 , 0.2867

In [10]:
EXP_NUM = 3
EXP_NAME = 'second_order_X'
arr_for_plotting = in_dict[EXP_NUM][EXP_NAME]
arr_for_plotting

array([0.23449796, 0.2345688 , 0.0989702 , 0.37882134, 0.02910594,
       0.27740997, 0.16379115, 0.19063637, 0.41707927, 0.29391664,
       0.1971741 , 0.21215774, 0.3162869 , 0.2859519 , 0.3024328 ,
       0.18131642, 0.4094162 , 0.2760841 , 0.23804706, 0.30866247,
       0.30504382, 0.30536607, 0.20394868, 0.3266883 , 0.28670394,
       0.314532  , 0.30773422, 0.38299367, 0.3626843 , 0.20989752,
       0.20443484, 0.2916711 , 0.32588214], dtype=float32)

# Sandbox

In [9]:
arr1 = np.array([[1],[2],[3]]).reshape(-1)
arr2 = np.array([[4],[5],[6]]).reshape(-1)
np.concatenate((arr1,arr2))

array([1, 2, 3, 4, 5, 6])

In [7]:
def new_save_file(filepath):
    dct = defaultdict()
    save_pickle(dct, filepath)

In [9]:
new_save_file(FILEPATH)